<a href="https://colab.research.google.com/github/VarishAdudodla/BreastCancerPrediction/blob/main/lstmweatherpred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
import numpy as np

In [ ]:
df = pd.read_csv("LSTM-Multivariate_pollution.csv")

In [ ]:
df

,date,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0
...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [ ]:
temperatures = df['temp'].values.reshape(-1,1)
scaler = StandardScaler()
temperatures_scaled = scaler.fit_transform(temperatures)
temperatures_scaled
temperatures_scaled[1:3]

array([[-1.34984913],
       [-1.43186175]])

In [ ]:
def CreateData(input, seq_length):
  X,y = [], []
  for i in range(len(input) - seq_length):
    X.append(input[i:i+seq_length])
    y.append(input[i+seq_length])
  return np.array(X), np.array(y)

X, y = CreateData(temperatures_scaled, 30)

In [ ]:
X.shape
y.shape
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
X.shape

/tmp/ipython-input-2662164134.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
/tmp/ipython-input-2662164134.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


torch.Size([43770, 30, 1])

In [ ]:
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True)

In [ ]:
class TemperaturePredictor(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.LSTM = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
    self.fc = nn.Linear(hidden_size, 1)

  def forward(self, x):
    output, (hn, cn) = self.LSTM(x)
    input_to_linear = output[:, -1, :]
    prediction = self.fc(input_to_linear)
    return prediction


In [ ]:
model = TemperaturePredictor(1, 64, 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
num_epochs = 50
for epoch in range(num_epochs):
  for x_batch, y_batch in dataloader:
    optimizer.zero_grad()
    outputs = model(x_batch)
    loss = criterion(outputs, y_batch)  # use the criterion here
    loss.backward()
    optimizer.step()
  print(f"Loss: {loss.item():.3f}")

Loss: 0.010
Loss: 0.017
Loss: 0.012
Loss: 0.006
Loss: 0.010
Loss: 0.007
Loss: 0.009
Loss: 0.008
Loss: 0.007
Loss: 0.007
Loss: 0.013
Loss: 0.006
Loss: 0.009
Loss: 0.009
Loss: 0.005
Loss: 0.004
Loss: 0.010
Loss: 0.009
Loss: 0.014
Loss: 0.009
Loss: 0.005
Loss: 0.005
Loss: 0.009
Loss: 0.011
Loss: 0.010
Loss: 0.007
Loss: 0.006
Loss: 0.009
Loss: 0.004
Loss: 0.010
Loss: 0.007
Loss: 0.016
Loss: 0.006
Loss: 0.014
Loss: 0.005
Loss: 0.011
Loss: 0.005
Loss: 0.009
Loss: 0.008
Loss: 0.006
Loss: 0.006
Loss: 0.005
Loss: 0.007
Loss: 0.004
Loss: 0.008
Loss: 0.005
Loss: 0.011
Loss: 0.006
Loss: 0.007
Loss: 0.005


In [ ]:
model.eval()
with torch.no_grad():
  last_seq = X[-1].unsqueeze(0)
  predicted_temp = model(last_seq)
  print(predicted_temp.item(), y[-1])


-1.3631072044372559 tensor([-1.2678])


In [ ]:
torch.save(model.state_dict(), "temperature_lstm.pth")